In [66]:
import pandas as pd
from Scripts.dirty_accuracy import injection
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [67]:
german = pd.read_csv("datasets/german.csv", sep=',')
german_list = injection(df_pandas=german, seed=10, name='german', name_class='Risk')
german_list.append(german)
german_list.reverse()
german

saved german-accuracy50%
saved german-accuracy60%
saved german-accuracy70%
saved german-accuracy80%
saved german-accuracy90%


,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
0,67,male,2,own,little,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,little,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,little,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,little,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad


german_list[0] is the cleanest (accuracy 100%), german_list[5] is the dirtiest (accuracy 50%)

In [68]:
def DTreeClassifier(X_train, y_train, X_test, y_test):
    K = []
    training = []
    test = []
    scores = {}

    for k in range(1, 20):
        clf = DecisionTreeClassifier()
        clf.fit(X_train, y_train)

        training_score = clf.score(X_train, y_train)
        test_score = clf.score(X_test, y_test)
        K.append(k)

        training.append(training_score)
        test.append(test_score)
        scores[k] = [training_score, test_score]

    return scores

Data conversion map to change all categorical columns into numerical values

In [69]:
def DCmap(df):
    dataset = df.copy()
    for col in dataset:
        if dataset[col].dtype == object:
            to_replace = dataset[col].unique()
            values = []
            for i in range (len(to_replace)):
                values.append(i)
            dataset[col] = dataset[col].replace(to_replace=to_replace, value=values)
    return dataset

In [72]:
mappedDF = DCmap(german)

,Age,Sex,Job,Housing,Saving_accounts,Checking_account,Credit_amount,Duration,Purpose,Risk
0,67,male,2,own,little,little,1169,6,radio/TV,good
1,22,female,2,own,little,moderate,5951,48,radio/TV,bad
2,49,male,1,own,little,little,2096,12,education,good
3,45,male,2,free,little,little,7882,42,furniture/equipment,good
4,53,male,2,free,little,little,4870,24,car,bad
...,...,...,...,...,...,...,...,...,...,...
995,31,female,1,own,little,little,1736,12,furniture/equipment,good
996,40,male,3,own,little,little,3857,30,car,good
997,38,male,2,own,little,little,804,12,radio/TV,good
998,23,male,2,free,little,little,1845,45,radio/TV,bad
